In [1]:
import pandas as pd
import numpy as np

X_train = pd.read_csv("/kaggle/input/melting-point/train.csv")
X_test = pd.read_csv("/kaggle/input/melting-point/test.csv")

print(X_train.shape)
print(X_test.shape)

(2662, 427)
(666, 426)


In [2]:
# checking for missing values
print(X_train.isnull().sum())

id           0
SMILES       0
Tm           0
Group 1      0
Group 2      0
            ..
Group 420    0
Group 421    0
Group 422    0
Group 423    0
Group 424    0
Length: 427, dtype: int64


In [3]:
# prepare X and y
X = X_train.drop(columns = ['id', 'SMILES', 'Tm'])
y = X_train['Tm']

# print(X.head())
# print(y.head())

In [4]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [5]:
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import cross_val_score
# from xgboost import XGBRegressor

# def get_score(n_estimators):
#     my_pipeline = Pipeline(steps=[
#         ('model', XGBRegressor(n_estimators=n_estimators, learning_rate=0.05) )
#     ])
#     scores = -1 * cross_val_score(my_pipeline, train_X, train_y,
#                               cv=5,
#                               scoring='neg_mean_absolute_error')
#     cv_mae = scores.mean()

#     # Fit on full training set
#     my_pipeline.fit(train_X, train_y)
    
#     # Validation score
#     valid_preds = my_pipeline.predict(X_valid)
#     valid_mae = mean_absolute_error(y_valid, valid_preds)
    
#     #print(f"CV MAE: {cv_mae:.4f}")
#     #print(f"Validation MAE: {valid_mae:.4f}")

#     return cv_mae, valid_mae

In [6]:
# Define the model
from xgboost import XGBRegressor

my_model_1 = XGBRegressor(n_estimators=15000, learning_rate=0.0125) 

# Fit the model
my_model_1.fit(train_X, train_y)

# Get predictions
predictions_1 = my_model_1.predict(val_X)

# Calculate MAE
from sklearn.metrics import mean_absolute_error
mae_1 =  mean_absolute_error(predictions_1, val_y)

# Uncomment to print MAE
print("Mean Absolute Error:", mae_1)

Mean Absolute Error: 33.8018574200428


In [7]:
# define model
from sklearn.linear_model import LinearRegression

# Create model
my_model_2 = LinearRegression()

# Fit (train) the model
my_model_2.fit(train_X, train_y)

# Make predictions
predictions_2 = my_model_2.predict(val_X)

# Calculate MAE
from sklearn.metrics import mean_absolute_error
mae_2 =  mean_absolute_error(predictions_2, val_y)

# Uncomment to print MAE
print("Mean Absolute Error:", mae_2)

Mean Absolute Error: 23001673.781206608


In [8]:
# define model
from sklearn.ensemble import RandomForestRegressor

# Create the model
my_model_3 = RandomForestRegressor(n_estimators=10000)

# Train the model
my_model_3.fit(train_X, train_y)

# Make predictions
predictions_3 = my_model_3.predict(val_X)

# Calculate MAE
from sklearn.metrics import mean_absolute_error
mae_3 =  mean_absolute_error(predictions_3, val_y)

# Uncomment to print MAE
print("Mean Absolute Error:", mae_3)

Mean Absolute Error: 37.26223390927009


In [9]:
# determining optimal n_estimators
estimators = [100, 500, 1000, 5000, 10000, 15000, 20000, 25000]

for i in estimators:
    model = XGBRegressor(n_estimators=i, learning_rate=0.0125) 

    # Fit the model
    model.fit(train_X, train_y)
    
    # Get predictions
    pred = model.predict(val_X)
    
    # Calculate MAE
    from sklearn.metrics import mean_absolute_error
    mae =  mean_absolute_error(pred, val_y)
    
    # Uncomment to print MAE
    print("Mean Absolute Error for ", i, " estimators: ", mae)

Mean Absolute Error for  100  estimators:  51.39433909575442
Mean Absolute Error for  500  estimators:  42.270545037074555
Mean Absolute Error for  1000  estimators:  39.1947064105923
Mean Absolute Error for  5000  estimators:  34.67239519473535
Mean Absolute Error for  10000  estimators:  33.90017600454935
Mean Absolute Error for  15000  estimators:  33.8018574200428
Mean Absolute Error for  20000  estimators:  34.092209088466845
Mean Absolute Error for  25000  estimators:  34.23597633404758


In [10]:
# final prediction
final_X_test = X_test.drop(columns = ['id', 'SMILES'])

final_preds = my_model_1.predict(final_X_test)

submission = pd.DataFrame({'id': X_test['id'], 'Tm': final_preds})
submission.to_csv('submission.csv', index=False)
display(submission.head())

,id,Tm
0,1022,331.355682
1,1146,295.351013
2,79,230.048843
3,2279,209.369827
4,1342,217.608643
